In [5]:
from pathlib import Path
import os

import pandas as pd
import numpy as np
from data_processing.data_processing import *

from sklearn import preprocessing
from sklearn import linear_model
from sklearn import metrics
from sklearn.pipeline import Pipeline

In [6]:
MAIN_PATH = Path(os.getcwd()).parent

## Load data

In [168]:
org_df = pd.read_csv(os.path.join(MAIN_PATH, "data", "data.csv"))
org_df = load(org_df)
org_df

,001fcx00211.pv,001fcx00221.pv,001fcx00231.pv,001fcx00241.pv,001fir01307.daca.pv,001fir01308.daca.pv,001fir01309.daca.pv,001fir01310.daca.pv,001fir01311.daca.pv,001fir01312.daca.pv,...,001uxm0rf02.daca.pv,001uxm0rf03.daca.pv,037tix00254.daca.pv,037tix00264.daca.pv,prazonka_fe,prazonka_s,prob_corg,prob_fe,prob_s,temp_zuz
2020-09-30 22:00:00+00:00,56.729077,54.724422,11.966905,22.493207,108.702362,112.280388,110.013796,108.761583,110.604901,104.500427,...,92.024078,92.405281,24.665309,24.526161,4.48,8.98,8.6,4.55,9.87,1297.0
2020-09-30 22:01:00+00:00,54.771942,54.734675,12.026410,22.593412,108.846612,112.334600,109.906645,108.879083,110.644046,104.371977,...,92.024206,92.405239,24.663982,24.526161,4.48,8.98,8.6,4.55,9.87,NaN
2020-09-30 22:02:00+00:00,54.695816,54.774163,11.953671,22.428933,108.827640,112.362508,109.970932,108.844765,110.621665,104.387912,...,92.024334,92.405196,24.662656,24.526161,4.48,8.98,8.6,4.55,9.87,NaN
2020-09-30 22:03:00+00:00,54.154394,54.985713,12.052065,22.335388,108.623473,112.386061,109.836948,108.717994,110.464245,104.418573,...,92.024462,92.405153,24.661329,24.526161,4.48,8.98,8.6,4.55,9.87,NaN
2020-09-30 22:04:00+00:00,54.693184,54.490742,12.027310,22.412620,108.746784,112.396792,109.966910,108.978605,110.603660,104.464328,...,92.024590,92.405110,24.660003,24.525475,4.48,8.98,8.6,4.55,9.87,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-31 22:55:00+00:00,57.882366,58.080133,10.056829,30.486496,101.111263,98.338021,102.332506,101.967828,100.651891,97.071359,...,98.121880,98.225985,24.460338,24.320295,6.18,8.89,7.8,6.65,12.47,NaN
2022-01-31 22:56:00+00:00,57.953225,57.931008,9.935828,29.830075,101.034501,98.362465,102.332450,101.981035,100.575884,97.066000,...,98.121319,98.226027,24.464324,24.322539,6.18,8.89,7.8,6.65,12.47,NaN
2022-01-31 22:57:00+00:00,58.186232,57.291012,10.014864,29.350514,101.251020,98.353221,102.312023,102.017297,100.621084,97.223686,...,98.120757,98.226070,24.468310,24.324782,6.18,8.89,7.8,6.65,12.47,NaN
2022-01-31 22:58:00+00:00,59.016838,58.014325,10.048125,29.496899,101.073264,98.354741,102.336123,101.967655,100.596174,97.050953,...,98.120196,98.226113,24.472296,24.327025,6.18,8.89,7.8,6.65,12.47,NaN


In [277]:
df = pd.DataFrame(index=org_df.index)

## Simple features based on correlation

In [278]:
# srednia po 'TEMP POD 2 WARSTWĄ WYMURÓWKI [°C]'
TIX1 = org_df.columns[org_df.columns.str.contains('001tix')].values
df['TIX1'] = org_df[TIX1].mean(axis='columns')

In [279]:
# srednia po 'WODY POWROTNE KOLEKTORÓW [°C]'
TIR = org_df.columns[org_df.columns.str.contains('tir')].values
df['TIR'] = org_df[TIR].mean(axis='columns')

In [280]:
# dodanie reg nadawy koncentratu liw i uśrednienie
LIW = org_df.get(['001fcx00211.pv', '001fcx00221.pv'])
df['LIW'] = LIW.mean(axis='columns')

In [281]:
# dodanie pozostałych reg nadawy
df['001fcx00231.pv'] = org_df['001fcx00231.pv']
df['001fcx00241.pv'] = org_df['001fcx00241.pv']

In [282]:
# sumaryczna moc cieplna
NIR = org_df.columns[org_df.columns.str.contains('nir')].values
df[NIR] = org_df[NIR]

In [283]:
# dodanie temp żużla
df["TEMP_ZUZ"] = org_df["temp_zuz"]

In [284]:
pr_df = df.copy()
prw_df = df.copy()
df = df.dropna(axis=0)
df.shape

(11377, 7)

In [285]:
df

,TIX1,TIR,LIW,001fcx00231.pv,001fcx00241.pv,001nir0szr0.daca.pv,TEMP_ZUZ
2020-09-30 22:00:00+00:00,418.990965,29.086390,55.726750,11.966905,22.493207,14.362428,1297.0
2020-09-30 23:00:00+00:00,418.911544,29.000256,55.318554,11.544775,22.702118,14.978720,1295.0
2020-10-01 00:00:00+00:00,418.820059,29.021386,54.830731,11.053689,20.013150,15.199544,1303.0
2020-10-01 00:30:00+00:00,418.777284,29.064571,55.112358,10.980221,19.902115,14.815123,1302.0
2020-10-01 01:00:00+00:00,418.735368,29.092727,55.378319,11.057164,21.069357,14.617263,1303.0
...,...,...,...,...,...,...,...
2022-01-31 18:00:00+00:00,418.284737,27.694067,59.040197,10.019018,32.147630,15.607105,1300.0
2022-01-31 19:00:00+00:00,418.138199,27.760225,58.903828,10.013093,31.719701,14.846545,1305.0
2022-01-31 20:00:00+00:00,418.012402,27.761558,59.333633,10.064700,30.158128,15.967954,1304.0
2022-01-31 21:00:00+00:00,417.939267,27.704117,58.256581,9.989189,28.000376,16.147269,1308.0


## Split data

In [286]:
train, val, test = split(df)

In [287]:
X_train, X_val, X_test = train.drop(["TEMP_ZUZ"], axis=1), val.drop(["TEMP_ZUZ"], axis=1), test.drop(["TEMP_ZUZ"], axis=1)
y_train, y_val, y_test = train["TEMP_ZUZ"], val["TEMP_ZUZ"], test["TEMP_ZUZ"]

## Ridge Regression

In [288]:
ridge_model = Pipeline([('normalization', preprocessing.StandardScaler()),
                        ('ridge', linear_model.Ridge(alpha=1, solver='cholesky'))])
ridge_model.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_model.predict(X_val))

74.99455470419595

In [326]:
ridge_model.named_steps['ridge'].coef_

array([1.04742917, 0.30767634, 0.5734355 , 0.42162442, 0.72724903,
       0.96895008])

## Polynomial Regression

In [289]:
poly_model = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=2)),
                      ('linear', linear_model.LinearRegression(fit_intercept=False))])
poly_model = poly_model.fit(X_train, y_train)
metrics.mean_squared_error(y_val, poly_model.predict(X_val))

68.5341466351266

In [290]:
poly_model2 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                      ('linear', linear_model.LinearRegression(fit_intercept=False))])
poly_model2 = poly_model2.fit(X_train, y_train)
metrics.mean_squared_error(y_val, poly_model2.predict(X_val))

72.87256470732088

In [291]:
poly_model3 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                        ('normalization', preprocessing.StandardScaler()),
                        ('linear', linear_model.LinearRegression(fit_intercept=False))])
poly_model3 = poly_model3.fit(X_train, y_train)
metrics.mean_squared_error(y_val, poly_model3.predict(X_val))

1698786.9091769422

## SGD Regressor

In [292]:
sgd_model1 = Pipeline([('normalization', preprocessing.StandardScaler()),
                      ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l2', learning_rate='adaptive'))])

In [293]:
sgd_model1 = sgd_model1.fit(X_train, y_train)

In [294]:
metrics.mean_squared_error(y_val, sgd_model1.predict(X_val))

75.0421514196493

In [295]:
sgd_model2 = Pipeline([('normalization', preprocessing.StandardScaler()),
                      ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l1', learning_rate='adaptive'))])
sgd_model2 = sgd_model2.fit(X_train, y_train)
metrics.mean_squared_error(y_val, sgd_model2.predict(X_val))

75.01410573769674

In [296]:
sgd_model3 = Pipeline([('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l1', learning_rate='adaptive'))])
sgd_model3 = sgd_model3.fit(X_train, y_train)
metrics.mean_squared_error(y_val, sgd_model3.predict(X_val))

3.814191173187635e+17

## SGD + Poly Regression

In [297]:
sgd_poly_model1 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=2)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l2', learning_rate='adaptive'))])
sgd_poly_model1 = sgd_poly_model1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, sgd_poly_model1.predict(X_val))

68.95916710291037

In [298]:
sgd_poly_model2 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l2', learning_rate='adaptive'))])
sgd_poly_model2 = sgd_poly_model1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, sgd_poly_model1.predict(X_val))

67.34987898669688

In [299]:
sgd_poly_model2 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=4)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l2', learning_rate='adaptive'))])
sgd_poly_model2 = sgd_poly_model1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, sgd_poly_model1.predict(X_val))

68.57640597569322

## Poly + Ridge Regression

In [300]:
ridge_poly_model1 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='cholesky'))])
ridge_poly_model1 = ridge_poly_model1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_poly_model1.predict(X_val))

65.14348801187221

In [301]:
ridge_poly_model1.named_steps["ridge"].coef_

array([ 0.00000000e+00,  4.52378443e+00, -3.34639251e+00, -1.46443514e+01,
        6.54924183e+00,  8.06344187e+00,  7.29393133e+00, -3.45271545e+00,
        4.62462115e+00,  2.08519068e+00, -3.23196353e+00,  4.90047130e-01,
        6.26613504e+00, -2.61180529e+00, -9.70093319e+00,  4.45941194e+00,
        2.44369455e+00,  3.40342934e+00, -1.11089381e+01,  1.85206721e+01,
        1.15188009e+01, -6.50797412e-01,  6.73237876e+00,  7.63035925e-01,
       -5.23864705e+00,  7.69326200e+00, -5.01917843e+00, -4.33534167e+00,
       -1.21203056e+01,  3.01603696e+00,  1.97802970e+01, -1.08569431e+01,
       -5.92569424e+00,  4.03401360e+00,  3.09834758e+00,  9.82259281e+00,
       -5.03100070e+00, -4.78530247e+00,  2.97508501e+00, -2.55675742e+00,
        5.96010352e+00,  3.19669564e+00,  5.51093237e+00, -2.93086124e+00,
       -3.45176572e+00, -5.78871389e+00, -5.22350002e+00, -1.11626106e+01,
       -8.85109585e+00, -4.71657526e-02, -4.47745189e+00,  4.87795036e+00,
        4.49671914e-01, -

In [302]:
ridge_poly_model2 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='saga'))])
ridge_poly_model2 = ridge_poly_model2.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_poly_model2.predict(X_val))

67.42240253383534

In [303]:
ridge_poly_model3 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=4)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='saga'))])
ridge_poly_model3 = ridge_poly_model3.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_poly_model3.predict(X_val))

66.0606248077321

In [304]:
ridge_poly_model4 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=5)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=1, solver='saga'))])
ridge_poly_model4 = ridge_poly_model4.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_poly_model4.predict(X_val))

65.56136931059262

## Spline + Ridge Regression

In [305]:
ridge_spline_model1 = Pipeline([('spline', preprocessing.SplineTransformer(n_knots=3)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='svd'))])
ridge_spline_model1 = ridge_spline_model1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_spline_model1.predict(X_val))

63.21317123812843

In [306]:
ridge_spline_model2 = Pipeline([('spline', preprocessing.SplineTransformer(n_knots=3, knots='quantile')),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='svd'))])
ridge_spline_model2 = ridge_spline_model2.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_spline_model2.predict(X_val))

64.7297476479715

## ElasticNet Regression

In [307]:
elastic_model1 = Pipeline([('normalization', preprocessing.StandardScaler()),
                           ('elasitc', linear_model.ElasticNet(max_iter=10000))])
elastic_model1 = elastic_model1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, elastic_model1.predict(X_val))

79.39085001063494

In [308]:
elastic_model2 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=4)),
                           ('normalization', preprocessing.StandardScaler()),
                           ('elasitc', linear_model.ElasticNet(max_iter=10000))])
elastic_model2 = elastic_model2.fit(X_train, y_train)
metrics.mean_squared_error(y_val, elastic_model2.predict(X_val))

77.63034965431582

## Bayessian Regression

In [309]:
bay_model = Pipeline([('normalization', preprocessing.StandardScaler()),
                        ('bayessian', linear_model.BayesianRidge(n_iter=1000))])
bay_model = bay_model.fit(X_train, y_train)
metrics.mean_squared_error(y_val, bay_model.predict(X_val))

75.01047167885115

In [310]:
bay_poly_model = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3, interaction_only=True)),
                           ('normalization', preprocessing.StandardScaler()),
                            ('bayessian', linear_model.BayesianRidge(n_iter=1000, verbose=True, tol=1e-5))])
bay_poly_model = bay_poly_model.fit(X_train, y_train)
metrics.mean_squared_error(y_val, bay_poly_model.predict(X_val))

Convergence after  78  iterations


63.72758658722974

In [311]:
bay_poly_model.named_steps["bayessian"].coef_

array([ 1.10902839e-11, -2.49349616e+01, -2.08381609e+01, -4.04711376e+01,
        1.22114798e+02, -3.01994000e+01,  4.95694392e+00,  3.17067357e+01,
        4.41584024e+01, -5.48520705e+01,  5.11932567e+01,  3.18330943e+01,
       -1.68196412e+02,  7.48579398e+01,  8.30916371e+01,  1.71717939e+01,
        1.70133791e+02,  9.45279874e+01,  1.43650482e+02, -1.58501506e+02,
       -2.42984038e+02, -1.10989518e+02,  1.54157811e+02, -1.43054055e+02,
       -9.05412684e+01, -4.34770432e+01, -2.29866841e+02, -8.46941092e+01,
       -1.41321665e+02,  1.65937394e+02,  2.19222222e+02,  7.05655899e+01,
        9.31638599e+01, -1.11421647e+01, -5.71040696e+00, -3.13959432e+01,
        2.37653244e+01,  2.12961193e+01, -2.09361380e+00, -2.19510870e+01,
        8.82647646e+00,  1.64476518e+01])

# Using all features, without average

### Split data

In [90]:
all_df = org_df.dropna()
all_df

,001fcx00211.pv,001fcx00221.pv,001fcx00231.pv,001fcx00241.pv,001fir01307.daca.pv,001fir01308.daca.pv,001fir01309.daca.pv,001fir01310.daca.pv,001fir01311.daca.pv,001fir01312.daca.pv,...,001uxm0rf02.daca.pv,001uxm0rf03.daca.pv,037tix00254.daca.pv,037tix00264.daca.pv,prazonka_fe,prazonka_s,prob_corg,prob_fe,prob_s,temp_zuz
2020-09-30 22:00:00+00:00,56.729077,54.724422,11.966905,22.493207,108.702362,112.280388,110.013796,108.761583,110.604901,104.500427,...,92.024078,92.405281,24.665309,24.526161,4.48,8.98,8.60,4.55,9.87,1297.0
2020-09-30 23:00:00+00:00,55.309363,55.327746,11.544775,22.702118,108.378922,112.237887,109.862477,109.128895,110.520359,104.410075,...,92.033540,92.402722,24.585718,24.448561,4.48,8.98,8.60,4.55,9.87,1295.0
2020-10-01 00:00:00+00:00,54.703949,54.957514,11.053689,20.013150,109.861116,112.517942,110.237388,109.214988,110.434876,104.657074,...,92.056562,92.400504,24.564115,24.484372,4.48,8.98,8.60,4.55,9.87,1303.0
2020-10-01 00:30:00+00:00,55.000000,55.224717,10.980221,19.902115,109.585113,112.805842,110.151395,108.958777,110.694621,104.680675,...,92.068073,92.403063,24.610243,24.517869,4.48,8.98,8.60,4.55,9.87,1302.0
2020-10-01 01:00:00+00:00,55.886746,54.869892,11.057164,21.069357,109.757104,112.602264,110.165194,109.178997,110.596014,104.695973,...,92.076607,92.405623,24.605002,24.488603,4.30,7.84,8.61,4.58,9.74,1303.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-31 18:00:00+00:00,59.180029,58.900365,10.019018,32.147630,103.049852,98.548160,102.593229,102.306346,100.762990,97.234765,...,98.130381,98.257987,24.401280,24.226479,6.38,9.12,7.74,6.06,11.26,1300.0
2022-01-31 19:00:00+00:00,58.895968,58.911688,10.013093,31.719701,102.570421,98.549435,102.571353,102.108130,100.757045,97.183075,...,98.131138,98.234942,24.301374,24.145683,6.38,9.12,7.74,6.06,11.26,1305.0
2022-01-31 20:00:00+00:00,59.073501,59.593765,10.064700,30.158128,102.197750,98.577139,102.375469,102.194845,100.746604,97.138241,...,98.130569,98.218519,24.326187,24.222134,6.38,9.12,7.74,6.06,11.26,1304.0
2022-01-31 21:00:00+00:00,57.774647,58.738514,9.989189,28.000376,102.266821,98.477571,102.374577,102.107050,100.649014,97.129910,...,98.130687,98.221079,24.290438,24.144540,6.38,9.12,7.74,6.06,11.26,1308.0


In [91]:
train_a, val_a, test_a = split(all_df)

In [92]:
X_train_a, X_val_a, X_test_a = train_a.drop(["temp_zuz"], axis=1), val_a.drop(["temp_zuz"], axis=1), test_a.drop(["temp_zuz"], axis=1)
y_train_a, y_val_a, y_test_a = train_a["temp_zuz"], val_a["temp_zuz"], test_a["temp_zuz"]

### Polynomial Regression

In [93]:
poly_model_all = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=2)),
                          ('linear', linear_model.LinearRegression(fit_intercept=False))])
poly_model_all = poly_model_all.fit(X_train_a, y_train_a)
metrics.mean_squared_error(y_val_a, poly_model_all.predict(X_val_a))

3668115.416767741

### Poly + SGD Regression

In [94]:
sgd_poly_model_all = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=2)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='elasticnet', learning_rate='adaptive'))])
sgd_poly_model_all = sgd_poly_model_all.fit(X_train_a, y_train_a)
metrics.mean_squared_error(y_val_a, sgd_poly_model_all.predict(X_val_a))

6.053012405180626e+21

Wniosek: zdecydowanie nie chcemy brać wszystkich feauterów

# Adding PR to features

In [312]:
# wrzucam o prażonkach i prob
PR = org_df.columns[org_df.columns.str.contains('pr')].values
pr_df[PR] = org_df[PR]

In [313]:
# split data
pr_df = pr_df.dropna(axis=0)
train, val, test = split(pr_df)
X_train, X_val, X_test = train.drop(["TEMP_ZUZ"], axis=1), val.drop(["TEMP_ZUZ"], axis=1), test.drop(["TEMP_ZUZ"], axis=1)
y_train, y_val, y_test = train["TEMP_ZUZ"], val["TEMP_ZUZ"], test["TEMP_ZUZ"]

X_train

In [314]:
y_train

2020-09-30 22:00:00+00:00    1297.0
2020-09-30 23:00:00+00:00    1295.0
2020-10-01 00:00:00+00:00    1303.0
2020-10-01 00:30:00+00:00    1302.0
2020-10-01 01:00:00+00:00    1303.0
                              ...  
2021-09-05 20:00:00+00:00    1306.0
2021-09-05 21:00:00+00:00    1303.0
2021-09-05 22:00:00+00:00    1301.0
2021-09-05 23:00:00+00:00    1304.0
2021-09-06 00:00:00+00:00    1306.0
Name: TEMP_ZUZ, Length: 7963, dtype: float64

### Poly + SGD Regression

In [315]:
sgd_poly_model_pr = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=2)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l2', learning_rate='adaptive'))])
sgd_poly_model_pr = sgd_poly_model_pr.fit(X_train, y_train)
metrics.mean_squared_error(y_val, sgd_poly_model_pr.predict(X_val))

94.25600288606293

In [316]:
sgd_poly_model_pr.named_steps["sgd"].coef_

array([  0.        ,  -3.10613098,   0.30544907,  11.08469911,
         7.13402769,  11.24981217,   7.37633354,   4.29377554,
         1.48101844,   8.60200021,   4.81222876,  -5.0078236 ,
        -8.57638121,  10.96059648,  12.36970553,  -5.59630135,
        -1.93962344,  10.19524644,  -2.28892481,   3.80005669,
         8.1086176 ,   1.83468592,  -1.15824709,  -7.52129456,
        -4.94580548,  10.82970587,  -4.67746896,   2.73090886,
        -6.48474231,  -2.54818013,  -7.1349323 ,  13.68647583,
         1.02622193, -11.16532513,  17.17525192,   8.75462355,
        14.81431277,  -1.67325248,   4.64417534, -22.94270281,
       -15.48630106,  13.72568428,  -5.25806877,  -6.30521415,
       -17.1470121 ,   6.73134894,   2.16096639, -17.93743213,
        -3.70379152,  14.09304809,  -0.92586856,  -8.65846471,
        -3.07615467,  -0.6261515 , -11.55718184,  -7.16447792,
        17.80279701, -14.1305293 ,   1.42814811,   3.81161773,
        -6.85721205, -11.12887326,   5.59373276,   5.78

### SGD Regression

In [317]:
sgd_model_pr = Pipeline([('normalization', preprocessing.StandardScaler()),
                       ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l2', learning_rate='adaptive'))])
sgd_model_pr = sgd_model_pr.fit(X_train, y_train)
metrics.mean_squared_error(y_val, sgd_model_pr.predict(X_val))

78.15621104536906

In [318]:
sgd_model_pr.named_steps["sgd"].coef_

array([ 1.08162723,  0.37484071,  0.87534352,  0.31390239,  0.55041121,
        0.67919064,  0.10870552, -0.05558672,  0.30574434,  0.06184773,
        0.49825389])

### Poly + Ridge Regression

In [319]:
ridge_poly_model_pr = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='sag'))])
ridge_poly_model_pr = ridge_poly_model_pr.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_poly_model_pr.predict(X_val))

79.30627179608496

In [100]:
ridge_poly_model_pr1 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='svd'))])
ridge_poly_model_pr1 = ridge_poly_model_pr1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_poly_model_pr1.predict(X_val))

77.6476214726276

In [103]:
ridge_poly_model_pr2 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='saga'))])
ridge_poly_model_pr2 = ridge_poly_model_pr2.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_poly_model_pr2.predict(X_val))

79.4660965356888

### Ridge Regression

In [104]:
ridge_model_pr1 = Pipeline([('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=1, solver='cholesky'))])
ridge_model_pr1 = ridge_model_pr1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_model_pr1.predict(X_val))

78.15592482417102

In [167]:
ridge_model_pr1.named_steps["ridge"].coef_

array([ 1.08209998,  0.3691561 , -0.7964524 ,  1.66777108,  0.25955482,
        0.68953828,  0.60649385,  0.07767323, -0.0602534 ,  0.30672109,
        0.08191634,  0.52413647])

In [105]:
ridge_model_pr2 = Pipeline([('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=1, solver='sag'))])
ridge_model_pr2 = ridge_model_pr2.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_model_pr2.predict(X_val))

78.15381925985238

In [106]:
ridge_model_pr3 = Pipeline([('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=1, solver='svd'))])
ridge_model_pr3 = ridge_model_pr3.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_model_pr3.predict(X_val))

78.15592482417102

# Add pr, wentylatory i moc cieplną

In [327]:
# wrzucam o prażonkach i prob
PR = org_df.columns[org_df.columns.str.contains('pr')].values
prw_df[PR] = org_df[PR]

In [328]:
# wentylatory + uśrednienie
UXM = org_df.columns[org_df.columns.str.contains('uxm')].values
prw_df['UXM'] = org_df[UXM].mean(axis='columns')

In [329]:
# dodać moc cieplną
NIR = org_df.columns[org_df.columns.str.contains('nir')].values
prw_df[NIR] = org_df[NIR]

In [330]:
# dodać temp na kol
TXI = org_df.columns[org_df.columns.str.contains('txi')].values
prw_df[TXI] = org_df[TXI]

In [331]:
# woda chłodząca
FIR = org_df.columns[org_df.columns.str.contains('fir')].values
prw_df['FIR'] = org_df[FIR].mean(axis='columns')

In [332]:
prw_df

,TIX1,TIR,LIW,001fcx00231.pv,001fcx00241.pv,001nir0szr0.daca.pv,TEMP_ZUZ,prazonka_fe,prazonka_s,prob_corg,prob_fe,prob_s,UXM,001txi01153.daca.pv,001txi01154.daca.pv,FIR
2020-09-30 22:00:00+00:00,418.990965,29.086390,55.726750,11.966905,22.493207,14.362428,1297.0,4.48,8.98,8.6,4.55,9.87,92.173271,19.126427,19.929581,113.360791
2020-09-30 22:01:00+00:00,418.990059,29.087996,54.753309,12.026410,22.593412,14.381825,NaN,4.48,8.98,8.6,4.55,9.87,92.173326,19.125526,19.942439,113.354389
2020-09-30 22:02:00+00:00,418.989153,29.094825,54.734990,11.953671,22.428933,14.359413,NaN,4.48,8.98,8.6,4.55,9.87,92.173381,19.124626,19.919802,113.383070
2020-09-30 22:03:00+00:00,418.988247,29.098936,54.570053,12.052065,22.335388,14.353036,NaN,4.48,8.98,8.6,4.55,9.87,92.173436,19.123726,19.861671,113.281268
2020-09-30 22:04:00+00:00,418.987341,29.096812,54.591963,12.027310,22.412620,14.392053,NaN,4.48,8.98,8.6,4.55,9.87,92.173490,19.122826,19.803541,113.380821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-31 22:55:00+00:00,417.800499,27.868889,57.981250,10.056829,30.486496,18.381908,NaN,6.18,8.89,7.8,6.65,12.47,98.121336,9.249736,9.702534,104.704581
2022-01-31 22:56:00+00:00,417.799414,27.868535,57.942117,9.935828,29.830075,18.224828,NaN,6.18,8.89,7.8,6.65,12.47,98.121163,9.238108,9.698717,104.694446
2022-01-31 22:57:00+00:00,417.798330,27.859336,57.738622,10.014864,29.350514,18.105621,NaN,6.18,8.89,7.8,6.65,12.47,98.120990,9.226480,9.694900,104.756259
2022-01-31 22:58:00+00:00,417.797246,27.857374,58.515581,10.048125,29.496899,17.976463,NaN,6.18,8.89,7.8,6.65,12.47,98.120817,9.214853,9.691082,104.692785


In [333]:
# split data
prw_df = prw_df.dropna(axis=0)
train, val, test = split(prw_df)
print(train.shape)
X_train, X_val, X_test = train.drop(["TEMP_ZUZ"], axis=1), val.drop(["TEMP_ZUZ"], axis=1), test.drop(["TEMP_ZUZ"], axis=1)
y_train, y_val, y_test = train["TEMP_ZUZ"], val["TEMP_ZUZ"], test["TEMP_ZUZ"]

(7963, 16)


In [334]:
X_train

,TIX1,TIR,LIW,001fcx00231.pv,001fcx00241.pv,001nir0szr0.daca.pv,prazonka_fe,prazonka_s,prob_corg,prob_fe,prob_s,UXM,001txi01153.daca.pv,001txi01154.daca.pv,FIR
2020-09-30 22:00:00+00:00,418.990965,29.086390,55.726750,11.966905,22.493207,14.362428,4.48,8.98,8.60,4.55,9.87,92.173271,19.126427,19.929581,113.360791
2020-09-30 23:00:00+00:00,418.911544,29.000256,55.318554,11.544775,22.702118,14.978720,4.48,8.98,8.60,4.55,9.87,92.188524,18.885216,19.536727,113.300487
2020-10-01 00:00:00+00:00,418.820059,29.021386,54.830731,11.053689,20.013150,15.199544,4.48,8.98,8.60,4.55,9.87,92.187308,18.612499,19.254154,113.658312
2020-10-01 00:30:00+00:00,418.777284,29.064571,55.112358,10.980221,19.902115,14.815123,4.48,8.98,8.60,4.55,9.87,92.195284,18.455667,19.196964,113.649439
2020-10-01 01:00:00+00:00,418.735368,29.092727,55.378319,11.057164,21.069357,14.617263,4.30,7.84,8.61,4.58,9.74,92.191375,18.075895,18.785391,113.667926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-05 20:00:00+00:00,427.223464,27.474773,54.550562,4.938091,24.345593,22.548988,4.11,4.74,8.48,5.54,11.24,97.272127,20.054366,20.598770,112.817374
2021-09-05 21:00:00+00:00,426.885975,27.742631,54.896322,4.930626,26.030218,23.326609,4.72,7.72,8.64,5.58,11.40,97.277412,19.182281,19.653056,112.822626
2021-09-05 22:00:00+00:00,426.514867,27.705729,55.738373,4.994519,25.914533,21.962930,4.72,7.72,8.64,5.58,11.40,97.270984,18.364518,18.825136,112.711963
2021-09-05 23:00:00+00:00,426.142533,27.594741,56.957987,4.999296,24.831633,21.440236,4.72,7.72,8.64,5.58,11.40,97.266249,17.437377,17.925498,112.996533


In [335]:
ridge_model_prw1 = Pipeline([('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=1, solver='cholesky'))])
ridge_model_prw1 = ridge_model_prw1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_model_prw1.predict(X_val))

74.56598971886858

In [336]:
ridge_model_prw1.named_steps["ridge"].coef_

array([-0.12487341,  0.64812008,  1.35896191, -0.73396068,  0.38025117,
        0.45904702,  0.04616988, -0.21504712,  0.59048814,  0.39728254,
        0.28056458,  1.92339696, -2.79680812,  2.62442283, -0.28696775])

In [337]:
ridge_poly_model_prw1 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=2)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='svd'))])
ridge_poly_model_prw1 = ridge_poly_model_prw1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_poly_model_prw1.predict(X_val))

76.73303105083961

In [338]:
ridge_poly_model_prw1.named_steps["ridge"].coef_

array([-3.87842962e-13, -7.06506440e+00,  1.15470920e+01,  1.09843433e+01,
       -4.19717658e+00,  1.04712722e+01,  4.81731820e+00, -1.70319709e+00,
        2.50368669e+00, -3.33780197e-01,  5.29613468e+00, -2.02282797e+00,
       -7.99850342e+00, -2.38503247e+00, -4.04970550e+00, -2.39144739e+00,
       -3.24787221e+00,  4.35600262e+00,  7.89646722e+00, -4.59968394e+00,
        1.69011047e+01, -5.50572433e+00, -3.88783520e+00,  7.33477282e+00,
        5.15616281e+00, -2.34030159e+00,  2.24057469e+00,  1.03242293e+01,
        5.12655618e-01, -4.79274519e-01,  1.50673620e+00, -5.11545698e+00,
       -6.85364178e+00, -7.34839569e+00, -1.32674893e+01, -5.37240071e+00,
        6.68363064e+00, -4.36499184e+00,  4.33602185e+00,  1.73186158e+00,
       -3.19325782e+00, -1.66779293e+01,  6.57147635e+00,  2.99459368e+00,
       -1.64682183e+00, -1.28669356e+01,  2.42279682e+01,  1.36586152e+01,
        9.24444558e+00, -6.64305358e+00,  1.74002717e+00, -3.10635053e+00,
       -1.16368829e+01,  